In [1]:
from selenium import webdriver
import selenium
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import numpy as np

# Crawling code

In [13]:
def crawling(game_num):
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(3)
    new = []
    driver.get(f'https://sports.daum.net/gamecenter/{game_num}/highlight')
    html = driver.page_source
    time.sleep(2)

    soup = BeautifulSoup(html, 'html.parser')
    pitch = soup.select('#gameResultPitcherRecordWrapper')
    hit = soup.select('#gameResultBatterRecordWrapper')

    overall = soup.select('#default_4 > div.gc_cont.gc_record > div')
    date =soup.select('#gameScoreboardWrap > div > div > div.info_game > span.txt_time')

    res_ov = making_overall(overall, date)
    home_p, away_p = making_pitch(pitch)
    home_t, away_t = making_hit(hit)
    team_p = making_team_pitch(res_ov, away_t, home_t, away_p, home_p)
    team_h = making_team_hit(res_ov, away_t, home_t)
    return home_p, away_p, home_t, away_t,res_ov, team_p, team_h


# Overall Data

In [3]:
def making_overall(data, date):
    res = []
    team_list = ['LG', 'SK', '삼성', '키움', '한화', 'NC', 'KIA', '두산', 'KT', '롯데']
    new2 = ['득점']
    title = ['팀']
    team_cnt = 0
    for j in data:
        new = j.text.split('\n')
    for i in new:
        if team_cnt ==2:
            res.append(title)
            team_cnt+=1

        if i in team_list:
            title.append(i)
            team_cnt+=1
            continue

        if i=='':
            continue
        if '(' in i or 'vs' in i or '더보기' in i:
            continue
        if len(new2) == 3:
            res.append(new2)
            new2 = []
        new2.append(i)
    over = pd.DataFrame(res)
    o = over.T
    date = date[0].text[:5].replace('.', '')
    o[10] = np.array(['날짜', date, date])
    headers_o = o.iloc[0]
    o  = pd.DataFrame(o.values[1:], columns=headers_o)
    o[['득점', '안타', '홈런', '타율', '탈삼진', '도루', '실책', '병살', '잔루']] = o[['득점', '안타', '홈런', '타율', '탈삼진', '도루', '실책', '병살', '잔루']].astype('float32')
    return o

# PITCH Data

In [4]:
def making_pitch(data):
    new2 = []
    team_list = ['LG', 'SK', '삼성', '키움', '한화', 'NC', 'KIA', '두산', 'KT', '롯데']
    for j in data:
        new = j.text.split('\n')
    temp = []
    count = 0
    team_cnt = 0


    for i in new:
        if i in team_list:
            team_cnt+=1

        if team_cnt == 2:
            away_p = pd.DataFrame(new2)
            headers_a = away_p.iloc[0]
            away_p  = pd.DataFrame(away_p.values[1:], columns=headers_a)
            away_p['이닝'] =away_p['이닝'].apply(inning_change)
            away_p[['타자', '투구수', '타수', '피안타', '피홈런', '탈삼진', '사사구', '실점', '자책', 'ERA']] = away_p[['타자', '투구수', '타수', '피안타', '피홈런', '탈삼진', '사사구', '실점', '자책', 'ERA']].astype('float64')
            new2 = []
            team_cnt+=1

        if i == '':
            continue
        if '기록' in i:
            continue
        if i in ['승', '패', '홀', '세']:
            continue
        count+=1
        if count != 13:
            temp.append(i)
        else:
            new2.append(temp)
            temp=[]
            count=0
    home_p = pd.DataFrame(new2)
    headers_h = home_p.iloc[0]
    home_p  = pd.DataFrame(home_p.values[1:], columns=headers_h)
    home_p['이닝'] =home_p['이닝'].apply(inning_change)
    home_p[['타자', '투구수', '타수', '피안타', '피홈런', '탈삼진', '사사구', '실점', '자책', 'ERA']] = home_p[['타자', '투구수', '타수', '피안타', '피홈런', '탈삼진', '사사구', '실점', '자책', 'ERA']].astype('float64')
    return away_p, home_p

In [5]:
def inning_change(x):
    inni = {'0 ⅓': 1/3, '1 ⅓': 4/3, '2 ⅓': 7/3, '3 ⅓': 10/3, '4 ⅓': 13/3, '5 ⅓': 16/3, '6 ⅓': 19/3, '7 ⅓': 22/3, '8 ⅓':25/3,\
           '0 ⅔':2/3, '1 ⅔':5/3, '2 ⅔':8/3, '3 ⅔':11/3, '4 ⅔':14/3, '5 ⅔':17/3, '6 ⅔':20/3, '7 ⅔':23/3, '8 ⅔':26/3,\
           '0': 0, '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9}
    x = inni[x]
    return x

In [6]:
def making_team_pitch(res_ov, away_t, home_t, away_p, home_p):
    team_code = {'LG': 'LG', 'SK': 'SK', 'KT':'KT', '한화': 'HH', '키움':'WO', '삼성': 'SS', '두산':'OB', 'KIA':'HT', '롯데':'LT', 'NC':'NC'}

    if res_ov['득점'][0] > res_ov['득점'][1]:
        aw = 'W'
        ho = 'L'
    elif res_ov['득점'][1] > res_ov['득점'][0]:
        aw = 'L'
        ho = 'W'
    else:
        aw = 'D'
        ho = 'D'

    aw_p = [int('20'+res_ov['날짜'][0]), team_code[res_ov['팀'][0]], team_code[res_ov['팀'][1]], 'T', aw, round(away_p['이닝'].sum())*3, away_p['투구수'].sum(),\
            home_t['타수'].sum(), home_t['안타'].sum(), home_t['2타'].sum(), home_t['3타'].sum(), home_t['홈런'].sum(),\
            res_ov['도루'][1], away_p['사사구'].sum(),away_p['탈삼진'].sum(), res_ov['병살'][1], away_p['실점'].sum(), away_p['자책'].sum()]  


    ho_p = [int('20'+res_ov['날짜'][0]), team_code[res_ov['팀'][1]], team_code[res_ov['팀'][0]], 'B', ho, round(home_p['이닝'].sum())*3, home_p['투구수'].sum(),\
            away_t['타수'].sum(), away_t['안타'].sum(), away_t['2타'].sum(), away_t['3타'].sum(), away_t['홈런'].sum(),\
            res_ov['도루'][0], home_p['사사구'].sum(),home_p['탈삼진'].sum(), res_ov['병살'][0], home_p['실점'].sum(), home_p['자책'].sum()]

    team_pitch = pd.DataFrame([ho_p, aw_p], columns=['GDAY_DS', 'T_ID', 'VS_T_ID', 'TB_SC', 'WLS', 'INN2', 'BF', 'AB','HIT', 'H2', 'H3', 'HR',\
                                                    'SB', 'BB', 'KK', 'GD', 'R', 'ER'])
    return team_pitch

# HIT Data

In [7]:
def making_hit(data):
    new2 = []
    team_list = ['LG', 'SK', '삼성', '키움', '한화', 'NC', 'KIA', '두산', 'KT', '롯데']
    for j in data:
        new = j.text.split('\n')
    temp = []
    count = 0
    team_cnt = 0


    for i in new:
        if i in team_list:
            team_cnt+=1
        if '(' in i:
            continue
        if team_cnt == 2:
            away_t = pd.DataFrame(new2)
            headers_a = away_t.iloc[0]
            away_t  = pd.DataFrame(away_t.values[1:], columns=headers_a)
            away_t[['타수', '안타', '2타', '3타', '홈런', '득점', '타점', '삼진', '사사구']] = away_t[['타수', '안타', '2타', '3타', '홈런', '득점', '타점', '삼진', '사사구']].astype('float64')
            new2 = []
            team_cnt+=1

        if i == '':
            continue
        if '기록' in i:
            continue
        if i in ['승', '패', '홀', '세']:
            continue
        count+=1
        if count != 11:
            temp.append(i)
        else:
            new2.append(temp)
            temp=[]
            count=0
    home_t = pd.DataFrame(new2)
    headers_h = home_t.iloc[0]
    home_t  = pd.DataFrame(home_t.values[1:], columns=headers_h)
    home_t[['타수', '안타', '2타', '3타', '홈런', '득점', '타점', '삼진', '사사구']] = home_t[['타수', '안타', '2타', '3타', '홈런', '득점', '타점', '삼진', '사사구']].astype('float64')
    
    return away_t, home_t

In [8]:
def making_team_hit(res_ov, away_t, home_t):
    team_code = {'LG': 'LG', 'SK': 'SK', 'KT':'KT', '한화': 'HH', '키움':'WO', '삼성': 'SS', '두산':'OB', 'KIA':'HT', '롯데':'LT', 'NC':'NC'}

    aw_t = [int('20'+res_ov['날짜'][0]), team_code[res_ov['팀'][0]], team_code[res_ov['팀'][1]], 'T', away_t['타수'].sum(), away_t['타점'].sum(),away_t['득점'].sum(),\
        away_t['안타'].sum(), away_t['2타'].sum(), away_t['3타'].sum(), away_t['홈런'].sum(), res_ov.iloc[0]['도루'], away_t['사사구'].sum(),\
        away_t['삼진'].sum(), res_ov.iloc[0]['병살'], res_ov.iloc[0]['실책'], res_ov.iloc[0]['잔루']]

    ho_t = [int('20'+res_ov['날짜'][0]), team_code[res_ov['팀'][1]], team_code[res_ov['팀'][0]], 'B', home_t['타수'].sum(), home_t['타점'].sum(),home_t['득점'].sum(),\
        home_t['안타'].sum(), home_t['2타'].sum(), home_t['3타'].sum(), home_t['홈런'].sum(), res_ov.iloc[1]['도루'], home_t['사사구'].sum(),\
        home_t['삼진'].sum(), res_ov.iloc[1]['병살'], res_ov.iloc[1]['실책'], res_ov.iloc[1]['잔루']]

    team_hit = pd.DataFrame([ho_t, aw_t], columns=['GDAY_DS', 'T_ID', 'VS_T_ID', 'TB_SC', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'BB', 'KK', 'GD', 'ERR', 'LOB'])
    return team_hit

# Individual stats code

In [9]:
def indiv_pitch(data, visit):
    global res_ov, team_code, pl_info
    if visit == 'h':
        me = team_code[res_ov['팀'][1]]
        versus = team_code[res_ov['팀'][0]]
        tb = 'B'
    else:
        me = team_code[res_ov['팀'][0]]
        versus = team_code[res_ov['팀'][1]]
        tb = 'T'
        
    data['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
    data['T_ID'] = np.array(me)
    data['VS_T_ID']  = np.array(versus)
    data['TB_SC'] = np.array(tb)
    name = []
    for i in data.iloc[:, 0]:
        name.append(pl_info[(pl_info['T_ID']==team_code[data.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
    data['P_ID'] =  np.array(name)
    data['START_CK'] = np.array([1]+[0]*(len(data)-1))
    data['RELIEF_CK'] = np.array([0]+[1]*(len(data)-1))
    data['INN2'] = round(data['이닝'] * 3)
    data = data.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                           '자책':'ER'}, axis='columns')
    data = data.iloc[:,2:]
    return data

# Main code

In [22]:
# team
team_pitch = pd.DataFrame(columns=['GDAY_DS', 'T_ID', 'VS_T_ID', 'TB_SC', 'WLS', 'INN2', 'BF', 'AB','HIT', 'H2', 'H3', 'HR',\
                                                    'SB', 'BB', 'KK', 'GD', 'R', 'ER'])

team_hit = pd.DataFrame(columns=['GDAY_DS', 'T_ID', 'VS_T_ID', 'TB_SC', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'BB', 'KK', 'GD', 'ERR', 'LOB'])
indiv_pitch = pd.DataFrame(columns=['GDAY_DS', 'T_ID', 'VS_T_ID', 'TB_SC', 'P_ID', 'START_CK', 'RELIEF_CK', 'INN2', 'BF', 'PA', 'AB', 'HIT', \
                                   'HR', 'KK', 'BB', 'R', 'ER', 'ERA'])
indiv_hit = pd.DataFrame(columns=['GDAY_DS', 'T_ID', 'VS_T_ID', 'TB_SC', 'P_ID', 'AB', 'RBI','RUN', 'HIT', 'H2', 'H3', 'HR', 'BB', 'KK'])


team_code = {'LG': 'LG', 'SK': 'SK', 'KT':'KT', '한화': 'HH', '키움':'WO', '삼성': 'SS', '두산':'OB', 'KIA':'HT', '롯데':'LT', 'NC':'NC'}
pl_info = pd.read_csv('player_info_2020.csv', encoding='cp949')

for i in range(80032844, 80033044):
    try:
        home_p, away_p, home_t, away_t,res_ov, team_p, team_h = crawling(i)
        team_pitch = pd.concat([team_pitch, team_p])
        team_hit = pd.concat([team_hit, team_h], ignore_index=True)
        
        
         # home_p
        home_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_p['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_p['TB_SC'] = np.array('B')
        name = []
        for i in home_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        home_p['P_ID'] =  np.array(name)
        home_p['START_CK'] = np.array([1]+[0]*(len(home_p)-1))
        home_p['RELIEF_CK'] = np.array([0]+[1]*(len(home_p)-1))
        home_p['INN2'] = round(home_p['이닝'] * 3)
        home_p = home_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        home_p = home_p.iloc[:,2:]


        #away_p

        away_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_p['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_p['TB_SC'] = np.array('T')
        name = []
        for i in away_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        away_p['P_ID'] =  np.array(name)
        away_p['START_CK'] = np.array([1]+[0]*(len(away_p)-1))
        away_p['RELIEF_CK'] = np.array([0]+[1]*(len(away_p)-1))
        away_p['INN2'] = round(away_p['이닝'] * 3)
        away_p = away_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        away_p = away_p.iloc[:,2:]




        # home_t
        home_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_t['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_t['TB_SC'] = np.array('B')
        name = []
        for i in home_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        home_t['P_ID'] =  np.array(name)
        home_t = home_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        home_t = home_t.iloc[:, 1:]



        # away_t
        away_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_t['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_t['TB_SC'] = np.array('T')
        name = []
        for i in away_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        away_t['P_ID'] =  np.array(name)
        away_t = away_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        away_t = away_t.iloc[:, 1:]







        indiv_pitch= pd.concat([indiv_pitch, home_p])
        indiv_pitch= pd.concat([indiv_pitch, away_p])
        indiv_hit= pd.concat([indiv_hit, home_t])
        indiv_hit= pd.concat([indiv_hit, away_t])
        
        
        
    except:
        pass
# team_pitch.to_csv('t1.csv', index=False)
# team_hit.to_csv('t2.csv', index=False)


In [20]:
team_pitch

,GDAY_DS,T_ID,VS_T_ID,TB_SC,WLS,INN2,BF,AB,HIT,H2,H3,HR,SB,BB,KK,GD,R,ER
0,200903,HH,WO,B,L,27.0,168.0,34.0,9.0,0.0,0.0,1.0,2.0,5.0,8.0,0.0,6.0,6.0
1,200903,WO,HH,T,W,27.0,157.0,38.0,11.0,2.0,0.0,1.0,0.0,6.0,8.0,0.0,5.0,3.0


In [21]:
team_hit

,GDAY_DS,T_ID,VS_T_ID,TB_SC,AB,RBI,RUN,HIT,H2,H3,HR,SB,BB,KK,GD,ERR,LOB
0,200903,HH,WO,B,38.0,3.0,5.0,11.0,2.0,0.0,1.0,0.0,6.0,8.0,0.0,0.0,12.0
1,200903,WO,HH,T,34.0,6.0,6.0,9.0,0.0,0.0,1.0,2.0,5.0,8.0,0.0,2.0,7.0


In [23]:
# 다른 경기들
for i in range(80037895, 80037980):
    try:
        home_p, away_p, home_t, away_t,res_ov, team_p, team_h = crawling(i)
        team_pitch = pd.concat([team_pitch, team_p])
        team_hit = pd.concat([team_hit, team_h], ignore_index=True)
        
        
         # home_p
        home_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_p['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_p['TB_SC'] = np.array('B')
        name = []
        for i in home_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        home_p['P_ID'] =  np.array(name)
        home_p['START_CK'] = np.array([1]+[0]*(len(home_p)-1))
        home_p['RELIEF_CK'] = np.array([0]+[1]*(len(home_p)-1))
        home_p['INN2'] = round(home_p['이닝'] * 3)
        home_p = home_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        home_p = home_p.iloc[:,2:]


        #away_p

        away_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_p['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_p['TB_SC'] = np.array('T')
        name = []
        for i in away_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        away_p['P_ID'] =  np.array(name)
        away_p['START_CK'] = np.array([1]+[0]*(len(away_p)-1))
        away_p['RELIEF_CK'] = np.array([0]+[1]*(len(away_p)-1))
        away_p['INN2'] = round(away_p['이닝'] * 3)
        away_p = away_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        away_p = away_p.iloc[:,2:]




        # home_t
        home_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_t['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_t['TB_SC'] = np.array('B')
        name = []
        for i in home_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        home_t['P_ID'] =  np.array(name)
        home_t = home_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        home_t = home_t.iloc[:, 1:]



        # away_t
        away_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_t['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_t['TB_SC'] = np.array('T')
        name = []
        for i in away_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        away_t['P_ID'] =  np.array(name)
        away_t = away_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        away_t = away_t.iloc[:, 1:]







        indiv_pitch= pd.concat([indiv_pitch, home_p])
        indiv_pitch= pd.concat([indiv_pitch, away_p])
        indiv_hit= pd.concat([indiv_hit, home_t])
        indiv_hit= pd.concat([indiv_hit, away_t])
        
        
        
    except:
        pass

In [24]:
# 더블헤더

for i in [80040561, 80041744, 80041745, 80041816, 80041817, 80041243, 80041244, 80041245, 80041246, 80042614, 80042613, 80041247, 80041248,\
         80041240, 80041241]:
    try:
        home_p, away_p, home_t, away_t,res_ov, team_p, team_h = crawling(i)
        team_pitch = pd.concat([team_pitch, team_p])
        team_hit = pd.concat([team_hit, team_h], ignore_index=True)
        
        
         # home_p
        home_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_p['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_p['TB_SC'] = np.array('B')
        name = []
        for i in home_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        home_p['P_ID'] =  np.array(name)
        home_p['START_CK'] = np.array([1]+[0]*(len(home_p)-1))
        home_p['RELIEF_CK'] = np.array([0]+[1]*(len(home_p)-1))
        home_p['INN2'] = round(home_p['이닝'] * 3)
        home_p = home_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        home_p = home_p.iloc[:,2:]


        #away_p

        away_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_p['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_p['TB_SC'] = np.array('T')
        name = []
        for i in away_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        away_p['P_ID'] =  np.array(name)
        away_p['START_CK'] = np.array([1]+[0]*(len(away_p)-1))
        away_p['RELIEF_CK'] = np.array([0]+[1]*(len(away_p)-1))
        away_p['INN2'] = round(away_p['이닝'] * 3)
        away_p = away_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        away_p = away_p.iloc[:,2:]




        # home_t
        home_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_t['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_t['TB_SC'] = np.array('B')
        name = []
        for i in home_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        home_t['P_ID'] =  np.array(name)
        home_t = home_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        home_t = home_t.iloc[:, 1:]



        # away_t
        away_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_t['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_t['TB_SC'] = np.array('T')
        name = []
        for i in away_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        away_t['P_ID'] =  np.array(name)
        away_t = away_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        away_t = away_t.iloc[:, 1:]







        indiv_pitch= pd.concat([indiv_pitch, home_p])
        indiv_pitch= pd.concat([indiv_pitch, away_p])
        indiv_hit= pd.concat([indiv_hit, home_t])
        indiv_hit= pd.concat([indiv_hit, away_t])
        
        
        
    except:
        pass

In [39]:
indiv_pitch.sort_values(by='GDAY_DS').to_csv('indiv_pitch.csv', index=False)

In [41]:
indiv_hit.sort_values(by='GDAY_DS').to_csv('indiv_hit.csv', index=False)

In [42]:
team_pitch.sort_values(by='GDAY_DS').to_csv('team_pitch.csv', index=False)

In [43]:
team_hit.sort_values(by='GDAY_DS').to_csv('team_hit.csv', index=False)

In [37]:
for i in [80032946]:
    try:
        home_p, away_p, home_t, away_t,res_ov, team_p, team_h = crawling(i)
        team_pitch = pd.concat([team_pitch, team_p])
        team_hit = pd.concat([team_hit, team_h], ignore_index=True)
        
        
         # home_p
        home_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_p['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_p['TB_SC'] = np.array('B')
        name = []
        for i in home_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        home_p['P_ID'] =  np.array(name)
        home_p['START_CK'] = np.array([1]+[0]*(len(home_p)-1))
        home_p['RELIEF_CK'] = np.array([0]+[1]*(len(home_p)-1))
        home_p['INN2'] = round(home_p['이닝'] * 3)
        home_p = home_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        home_p = home_p.iloc[:,2:]


        #away_p

        away_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_p['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_p['TB_SC'] = np.array('T')
        name = []
        for i in away_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        away_p['P_ID'] =  np.array(name)
        away_p['START_CK'] = np.array([1]+[0]*(len(away_p)-1))
        away_p['RELIEF_CK'] = np.array([0]+[1]*(len(away_p)-1))
        away_p['INN2'] = round(away_p['이닝'] * 3)
        away_p = away_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        away_p = away_p.iloc[:,2:]




        # home_t
        home_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_t['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_t['TB_SC'] = np.array('B')
        name = []
        for i in home_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        home_t['P_ID'] =  np.array(name)
        home_t = home_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        home_t = home_t.iloc[:, 1:]



        # away_t
        away_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_t['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_t['TB_SC'] = np.array('T')
        name = []
        for i in away_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        away_t['P_ID'] =  np.array(name)
        away_t = away_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        away_t = away_t.iloc[:, 1:]







        indiv_pitch= pd.concat([indiv_pitch, home_p])
        indiv_pitch= pd.concat([indiv_pitch, away_p])
        indiv_hit= pd.concat([indiv_hit, home_t])
        indiv_hit= pd.concat([indiv_hit, away_t])
        
        
        
    except:
        pass

In [47]:
indiv_hit

,GDAY_DS,T_ID,VS_T_ID,TB_SC,P_ID,AB,RBI,RUN,HIT,H2,H3,HR,BB,KK
0,200721,HH,HT,B,74163,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,200721,HH,HT,B,68743,3.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,200721,HH,HT,B,62700,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,200721,HH,HT,B,50730,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,200721,HH,HT,B,50709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,200901,LT,KT,T,79608,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,200901,LT,KT,T,62556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10,200901,LT,KT,T,76100,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11,200901,LT,KT,T,62558,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
# 7월 23일부터 8월 초

for i in range(80041744, 80041746):
    try:
        home_p, away_p, home_t, away_t, res_ov = crawling(i)

        # home_p
        home_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_p['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_p['TB_SC'] = np.array('B')
        name = []
        for i in home_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        home_p['P_ID'] =  np.array(name)
        home_p['START_CK'] = np.array([1]+[0]*(len(home_p)-1))
        home_p['RELIEF_CK'] = np.array([0]+[1]*(len(home_p)-1))
        home_p['INN2'] = round(home_p['이닝'] * 3)
        home_p = home_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        home_p = home_p.iloc[:,2:]


        #away_p

        away_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_p['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_p['TB_SC'] = np.array('T')
        name = []
        for i in away_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        away_p['P_ID'] =  np.array(name)
        away_p['START_CK'] = np.array([1]+[0]*(len(away_p)-1))
        away_p['RELIEF_CK'] = np.array([0]+[1]*(len(away_p)-1))
        away_p['INN2'] = round(away_p['이닝'] * 3)
        away_p = away_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        away_p = away_p.iloc[:,2:]




        # home_t
        home_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_t['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_t['TB_SC'] = np.array('B')
        name = []
        for i in home_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        home_t['P_ID'] =  np.array(name)
        home_t = home_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        home_t = home_t.iloc[:, 1:]



        # away_t
        away_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_t['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_t['TB_SC'] = np.array('T')
        name = []
        for i in away_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        away_t['P_ID'] =  np.array(name)
        away_t = away_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        away_t = away_t.iloc[:, 1:]



        indiv_pitch= pd.concat([indiv_pitch, home_p])
        indiv_pitch= pd.concat([indiv_pitch, away_p])
        indiv_hit= pd.concat([indiv_hit, home_t])
        indiv_hit= pd.concat([indiv_hit, away_t])
        
        
        # 팀기록 코드
#         team_pitch = pd.concat([team_pitch, team_p])
#         team_hit = pd.concat([team_hit, team_h], ignore_index=True)
    except:
        pass

In [ ]:
# 8월 30일 기아 KT 더블헤더

for i in range(80041816, 80041818):
    try:
        home_p, away_p, home_t, away_t, res_ov = crawling(i)

        # home_p
        home_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_p['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_p['TB_SC'] = np.array('B')
        name = []
        for i in home_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        home_p['P_ID'] =  np.array(name)
        home_p['START_CK'] = np.array([1]+[0]*(len(home_p)-1))
        home_p['RELIEF_CK'] = np.array([0]+[1]*(len(home_p)-1))
        home_p['INN2'] = round(home_p['이닝'] * 3)
        home_p = home_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        home_p = home_p.iloc[:,2:]


        #away_p

        away_p['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_p['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_p['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_p['TB_SC'] = np.array('T')
        name = []
        for i in away_p.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_p.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')
        away_p['P_ID'] =  np.array(name)
        away_p['START_CK'] = np.array([1]+[0]*(len(away_p)-1))
        away_p['RELIEF_CK'] = np.array([0]+[1]*(len(away_p)-1))
        away_p['INN2'] = round(away_p['이닝'] * 3)
        away_p = away_p.rename({'투구수': 'BF', '타자':'PA', '타수':'AB', '피안타':'HIT', '피홈런':'HR', '탈삼진':'KK', '사사구':'BB', '실점':'R',\
                               '자책':'ER'}, axis='columns')
        away_p = away_p.iloc[:,2:]




        # home_t
        home_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        home_t['T_ID'] = np.array(team_code[res_ov['팀'][1]])
        home_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][0]])
        home_t['TB_SC'] = np.array('B')
        name = []
        for i in home_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[home_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        home_t['P_ID'] =  np.array(name)
        home_t = home_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        home_t = home_t.iloc[:, 1:]



        # away_t
        away_t['GDAY_DS'] = np.array(int('20'+res_ov['날짜'][0]))
        away_t['T_ID'] = np.array(team_code[res_ov['팀'][0]])
        away_t['VS_T_ID']  = np.array(team_code[res_ov['팀'][1]])
        away_t['TB_SC'] = np.array('T')
        name = []
        for i in away_t.iloc[:, 0]:
            try:
                name.append(pl_info[(pl_info['T_ID']==team_code[away_t.columns[0]])& (pl_info['NAME']==i)].iloc[0]['PCODE'])
            except:
                name.append('No_Code')

        away_t['P_ID'] =  np.array(name)
        away_t = away_t.rename({'타수':'AB', '타점':'RBI', '득점':'RUN', '안타':'HIT', '2타':'H2', '3타':'H3', '홈런':'HR', '사사구':'BB',\
                               '삼진':'KK'}, axis='columns')
        away_t = away_t.iloc[:, 1:]



        indiv_pitch= pd.concat([indiv_pitch, home_p])
        indiv_pitch= pd.concat([indiv_pitch, away_p])
        indiv_hit= pd.concat([indiv_hit, home_t])
        indiv_hit= pd.concat([indiv_hit, away_t])
        
        
        # 팀기록 코드
#         team_pitch = pd.concat([team_pitch, team_p])
#         team_hit = pd.concat([team_hit, team_h], ignore_index=True)
    except:
        pass

In [17]:
indiv_pitch.tail(30)

,GDAY_DS,T_ID,VS_T_ID,TB_SC,P_ID,START_CK,RELIEF_CK,INN2,BF,PA,AB,HIT,HR,KK,BB,R,ER,ERA
2,200721,SS,NC,T,62920,0,1,3.0,11.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,200721,SS,NC,T,73117,0,1,3.0,16.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00
4,200721,SS,NC,T,67421,0,1,3.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.00
5,200721,SS,NC,T,68403,0,1,1.0,12.0,4.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,27.00
0,200721,OB,WO,B,69045,1,0,21.0,94.0,25.0,25.0,5.0,0.0,8.0,0.0,0.0,0.0,0.00
1,200721,OB,WO,B,65241,0,1,3.0,20.0,5.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.00
2,200721,OB,WO,B,67246,0,1,1.0,19.0,4.0,3.0,2.0,0.0,1.0,1.0,1.0,1.0,27.00
3,200721,OB,WO,B,76329,0,1,2.0,10.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
0,200721,WO,OB,T,69343,1,0,17.0,91.0,24.0,23.0,6.0,1.0,4.0,1.0,6.0,6.0,9.53
1,200721,WO,OB,T,63348,0,1,1.0,7.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.00


In [18]:
indiv_hit.tail(30)

,GDAY_DS,T_ID,VS_T_ID,TB_SC,P_ID,AB,RBI,RUN,HIT,H2,H3,HR,BB,KK
7,200721,WO,OB,T,67304,4.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8,200721,WO,OB,T,65586,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9,200721,WO,OB,T,65362,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,200721,WO,OB,T,64657,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
11,200721,WO,OB,T,50357,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,200721,WO,OB,T,69306,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,200722,HH,HT,B,74163,2.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0
1,200722,HH,HT,B,74729,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
2,200722,HH,HT,B,50730,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,200722,HH,HT,B,71752,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [13]:
indiv_pitch.to_csv('indiv_pitch.csv', index=False)

In [14]:
indiv_hit.to_csv('indiv_hit.csv', index=False)